In [1]:
# Dependencies 
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly.express as px
import nfl_data_py as nfl 
from api_keys import api_key

In [2]:
# Open the CSV files
scores_data = pd.read_csv("spreadspoke_scores.csv")
games_data = pd.read_csv("Super_Grp_1_Proj_3/penalties/games.csv")
log_data = pd.read_csv("Super_Grp_1_Proj_3/penalties/log.csv")
penalties_data = pd.read_csv("Super_Grp_1_Proj_3/penalties/penalties.csv")
players_data = pd.read_csv("Super_Grp_1_Proj_3/penalties/players.csv")
team_data = pd.read_csv("Super_Grp_1_Proj_3/penalties/team.csv")

pd.set_option('display.max_columns', None)


In [3]:
# only use data from 09/10/2009 to 02/12/2023
# Assuming "schedule_date" is a string column, convert it to a datetime object
scores_data['schedule_date'] = pd.to_datetime(scores_data['schedule_date'])

# Define the date range
start_date = '2009-09-10'
end_date = '2023-02-12'

# Create a mask for the date range
date_mask = (scores_data['schedule_date'] >= start_date) & (scores_data['schedule_date'] <= end_date)

# Apply the mask to filter the DataFrame
filtered_scores_data = scores_data[date_mask]

# Now, filtered_scores_data contains the games within the specified date range
filtered_scores_data

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
9741,2009-09-10,2009,1,False,Pittsburgh Steelers,13.0,10.0,Tennessee Titans,PIT,-6.5,35,Heinz Field,False,67.0,9.0,64.0,NaN
9742,2009-09-13,2009,1,False,Arizona Cardinals,16.0,20.0,San Francisco 49ers,ARI,-4.5,45,University of Phoenix Stadium,False,72.0,0.0,NaN,indoor
9743,2009-09-13,2009,1,False,Atlanta Falcons,19.0,7.0,Miami Dolphins,ATL,-4.0,44.5,Georgia Dome,False,72.0,0.0,NaN,indoor
9744,2009-09-13,2009,1,False,Baltimore Ravens,38.0,24.0,Kansas City Chiefs,BAL,-13.0,36.5,M&T Bank Stadium,False,69.0,7.0,74.0,NaN
9745,2009-09-13,2009,1,False,Carolina Panthers,10.0,38.0,Philadelphia Eagles,PHI,-2.5,43.5,Bank of America Stadium,False,77.0,7.0,59.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13511,2023-01-22,2022,Division,True,Buffalo Bills,10.0,27.0,Cincinnati Bengals,BUF,-6.0,48.5,Highmark Stadium,False,32.0,4.0,100.0,snow
13512,2023-01-22,2022,Division,True,San Francisco 49ers,19.0,12.0,Dallas Cowboys,SF,-3.5,46.5,Levi's Stadium,False,55.0,19.0,47.0,NaN
13513,2023-01-29,2022,Conference,True,Kansas City Chiefs,23.0,20.0,Cincinnati Bengals,KC,-1.5,48,GEHA Field at Arrowhead Stadium,False,22.0,13.0,55.0,NaN
13514,2023-01-29,2022,Conference,True,Philadelphia Eagles,31.0,7.0,San Francisco 49ers,PHI,-2.5,45.5,Lincoln Financial Field,False,52.0,14.0,48.0,rain


In [4]:
# create a new column to calculate the total scores of the game and compare to the over/under line
filtered_scores_data['total_score'] = filtered_scores_data['score_home'] + filtered_scores_data['score_away']

# Convert the over_under_line to a numerical data type
filtered_scores_data.loc[:,'over_under_line'] = pd.to_numeric(filtered_scores_data['over_under_line'], errors='coerce')

filtered_scores_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3775 entries, 9741 to 13515
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   schedule_date        3775 non-null   datetime64[ns]
 1   schedule_season      3775 non-null   int64         
 2   schedule_week        3775 non-null   object        
 3   schedule_playoff     3775 non-null   bool          
 4   team_home            3775 non-null   object        
 5   score_home           3775 non-null   float64       
 6   score_away           3775 non-null   float64       
 7   team_away            3775 non-null   object        
 8   team_favorite_id     3775 non-null   object        
 9   spread_favorite      3775 non-null   float64       
 10  over_under_line      3775 non-null   object        
 11  stadium              3775 non-null   object        
 12  stadium_neutral      3775 non-null   bool          
 13  weather_temperature  3059 non-null

/var/folders/7d/5c2gqmrn4mg4hc18ff5mjdw00000gn/T/ipykernel_64374/2672637089.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_scores_data['total_score'] = filtered_scores_data['score_home'] + filtered_scores_data['score_away']


In [5]:
draft_picks_df = nfl.import_draft_picks(years=[2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023])
draft_picks_df.info()
draft_picks_df

<class 'pandas.core.frame.DataFrame'>
Index: 3835 entries, 8321 to 12155
Data columns (total 36 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   season            3835 non-null   int32  
 1   round             3835 non-null   int32  
 2   pick              3835 non-null   int32  
 3   team              3835 non-null   object 
 4   gsis_id           2411 non-null   object 
 5   pfr_player_id     3817 non-null   object 
 6   cfb_player_id     3417 non-null   object 
 7   pfr_player_name   3835 non-null   object 
 8   hof               3835 non-null   bool   
 9   position          3835 non-null   object 
 10  category          3835 non-null   object 
 11  side              3835 non-null   object 
 12  college           3835 non-null   object 
 13  age               3811 non-null   float64
 14  to                3532 non-null   float64
 15  allpro            3835 non-null   int32  
 16  probowls          3835 non-null   int32  
 

,season,round,pick,team,gsis_id,pfr_player_id,cfb_player_id,pfr_player_name,hof,position,category,side,college,age,to,allpro,probowls,seasons_started,w_av,car_av,dr_av,games,pass_completions,pass_attempts,pass_yards,pass_tds,pass_ints,rush_atts,rush_yards,rush_tds,receptions,rec_yards,rec_tds,def_solo_tackles,def_ints,def_sacks
8321,2009,1,1,DET,00-0026498,StafMa00,matthew-stafford-1,Matthew Stafford,False,QB,QB,O,Georgia,21.0,2023.0,0,2,13,121.0,None,98.0,206.0,4834.0,7649.0,56047.0,357.0,180.0,411.0,1315.0,15.0,2.0,-3.0,0.0,1.0,NaN,NaN
8322,2009,1,2,STL,None,SmitJa22,jason-smith-3,Jason Smith,False,T,OL,O,Baylor,23.0,2012.0,0,0,2,10.0,None,9.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
8323,2009,1,3,KAN,None,JackTy98,tyson-jackson-1,Tyson Jackson,False,DE,DL,D,LSU,23.0,2016.0,0,0,7,34.0,None,23.0,122.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,167.0,NaN,9.0
8324,2009,1,4,SEA,None,CurrAa99,aaron-curry-1,Aaron Curry,False,LB,LB,D,Wake Forest,23.0,2012.0,0,0,3,16.0,None,12.0,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,163.0,NaN,5.5
8325,2009,1,5,NYJ,00-0026898,SancMa00,mark-sanchez-1,Mark Sanchez,False,QB,QB,O,USC,22.0,2018.0,0,0,4,32.0,None,26.0,79.0,1314.0,2320.0,15357.0,86.0,89.0,170.0,457.0,13.0,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12151,2023,7,255,SFO,00-0038648,GrahJa02,jalen-graham-1,Jalen Graham,False,S,DB,D,Purdue,22.0,2023.0,0,0,0,0.0,None,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN
12152,2023,7,256,GNB,None,DuboGr00,grant-dubose-1,Grant Dubose,False,WR,WR,O,Charlotte,22.0,NaN,0,0,0,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12153,2023,7,257,DEN,None,ForsAl00,alex-forsyth-1,Alex Forsyth,False,OL,OL,O,Oregon,24.0,NaN,0,0,0,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12154,2023,7,258,CHI,None,WillKe07,kendall-williamson-1,Kendall Williamson,False,S,DB,D,Stanford,23.0,NaN,0,0,0,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Filter data for 'O' side
df_offense = draft_picks_df[draft_picks_df['side'] == 'O']

# Filter data for 'D' side
df_defense = draft_picks_df[draft_picks_df['side'] == 'D']

# Display information for the two DataFrames
df_offense.info()
df_defense.info()

# Display the DataFrames
df_offense.head()  
df_defense.head()  


<class 'pandas.core.frame.DataFrame'>
Index: 1841 entries, 8321 to 12153
Data columns (total 36 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   season            1841 non-null   int32  
 1   round             1841 non-null   int32  
 2   pick              1841 non-null   int32  
 3   team              1841 non-null   object 
 4   gsis_id           1031 non-null   object 
 5   pfr_player_id     1835 non-null   object 
 6   cfb_player_id     1608 non-null   object 
 7   pfr_player_name   1841 non-null   object 
 8   hof               1841 non-null   bool   
 9   position          1841 non-null   object 
 10  category          1841 non-null   object 
 11  side              1841 non-null   object 
 12  college           1841 non-null   object 
 13  age               1833 non-null   float64
 14  to                1664 non-null   float64
 15  allpro            1841 non-null   int32  
 16  probowls          1841 non-null   int32  
 

,season,round,pick,team,gsis_id,pfr_player_id,cfb_player_id,pfr_player_name,hof,position,category,side,college,age,to,allpro,probowls,seasons_started,w_av,car_av,dr_av,games,pass_completions,pass_attempts,pass_yards,pass_tds,pass_ints,rush_atts,rush_yards,rush_tds,receptions,rec_yards,rec_tds,def_solo_tackles,def_ints,def_sacks
8323,2009,1,3,KAN,None,JackTy98,tyson-jackson-1,Tyson Jackson,False,DE,DL,D,LSU,23.0,2016.0,0,0,7,34.0,None,23.0,122.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,167.0,NaN,9.0
8324,2009,1,4,SEA,None,CurrAa99,aaron-curry-1,Aaron Curry,False,LB,LB,D,Wake Forest,23.0,2012.0,0,0,3,16.0,None,12.0,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,163.0,NaN,5.5
8329,2009,1,9,GNB,None,RajiBJ99,bj-raji-1,B.J. Raji,False,DT,DL,D,Boston Col.,23.0,2015.0,0,1,5,33.0,None,33.0,91.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,103.0,NaN,11.0
8331,2009,1,11,BUF,None,MaybAa99,aaron-maybin-1,Aaron Maybin,False,DE,DL,D,Penn St.,21.0,2012.0,0,0,0,5.0,None,2.0,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,NaN,6.0
8333,2009,1,13,WAS,00-0026989,OrakBr99,brian-orakpo-1,Brian Orakpo,False,LB,LB,D,Texas,23.0,2018.0,0,4,8,58.0,None,33.0,132.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,294.0,1.0,66.0


In [7]:
# Define the columns to keep
cols = ['season', 'round', 'pick', 'team', 'gsis_id', 'pfr_player_name', 'hof', 'position', 'college', 'age', 'to', 'probowls', 'seasons_started', 'games']

# Filter offense_df and defense_df based on desired columns
filtered_offense_df = df_offense[cols]
filtered_defense_df = df_defense[cols]

# Display the information of the filtered DataFrames
filtered_offense_df.info()
filtered_offense_df

filtered_defense_df.info()
filtered_defense_df

<class 'pandas.core.frame.DataFrame'>
Index: 1841 entries, 8321 to 12153
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   season           1841 non-null   int32  
 1   round            1841 non-null   int32  
 2   pick             1841 non-null   int32  
 3   team             1841 non-null   object 
 4   gsis_id          1031 non-null   object 
 5   pfr_player_name  1841 non-null   object 
 6   hof              1841 non-null   bool   
 7   position         1841 non-null   object 
 8   college          1841 non-null   object 
 9   age              1833 non-null   float64
 10  to               1664 non-null   float64
 11  probowls         1841 non-null   int32  
 12  seasons_started  1841 non-null   int32  
 13  games            1664 non-null   float64
dtypes: bool(1), float64(3), int32(5), object(5)
memory usage: 167.2+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 1927 entries, 8323 to 12155
Data columns

,season,round,pick,team,gsis_id,pfr_player_name,hof,position,college,age,to,probowls,seasons_started,games
8323,2009,1,3,KAN,None,Tyson Jackson,False,DE,LSU,23.0,2016.0,0,7,122.0
8324,2009,1,4,SEA,None,Aaron Curry,False,LB,Wake Forest,23.0,2012.0,0,3,48.0
8329,2009,1,9,GNB,None,B.J. Raji,False,DT,Boston Col.,23.0,2015.0,1,5,91.0
8331,2009,1,11,BUF,None,Aaron Maybin,False,DE,Penn St.,21.0,2012.0,0,0,48.0
8333,2009,1,13,WAS,00-0026989,Brian Orakpo,False,LB,Texas,23.0,2018.0,4,8,132.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12148,2023,7,252,BUF,00-0038646,Alex Austin,False,DB,Oregon St.,22.0,2023.0,0,0,8.0
12150,2023,7,254,NYG,None,Gervarrius Owens,False,DB,Houston,23.0,2023.0,0,0,3.0
12151,2023,7,255,SFO,00-0038648,Jalen Graham,False,S,Purdue,22.0,2023.0,0,0,4.0
12154,2023,7,258,CHI,None,Kendall Williamson,False,S,Stanford,23.0,NaN,0,0,NaN


In [8]:
# Drop rows where gsis_id is None
filtered_offense_df = filtered_offense_df[filtered_offense_df['gsis_id'].notna()]
filtered_defense_df = filtered_defense_df[filtered_defense_df['gsis_id'].notna()]

# Display the information of the updated DataFrame
filtered_offense_df.info()
filtered_offense_df

filtered_defense_df.info()
filtered_defense_df

<class 'pandas.core.frame.DataFrame'>
Index: 1031 entries, 8321 to 12149
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   season           1031 non-null   int32  
 1   round            1031 non-null   int32  
 2   pick             1031 non-null   int32  
 3   team             1031 non-null   object 
 4   gsis_id          1031 non-null   object 
 5   pfr_player_name  1031 non-null   object 
 6   hof              1031 non-null   bool   
 7   position         1031 non-null   object 
 8   college          1031 non-null   object 
 9   age              1031 non-null   float64
 10  to               983 non-null    float64
 11  probowls         1031 non-null   int32  
 12  seasons_started  1031 non-null   int32  
 13  games            983 non-null    float64
dtypes: bool(1), float64(3), int32(5), object(5)
memory usage: 93.6+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 1340 entries, 8333 to 12155
Data columns 

,season,round,pick,team,gsis_id,pfr_player_name,hof,position,college,age,to,probowls,seasons_started,games
8333,2009,1,13,WAS,00-0026989,Brian Orakpo,False,LB,Texas,23.0,2018.0,4,8,132.0
8334,2009,1,14,NOR,00-0026990,Malcolm Jenkins,False,DB,Ohio St.,21.0,2021.0,3,12,199.0
8345,2009,1,25,MIA,00-0027001,Vontae Davis,False,DB,Illinois,21.0,2018.0,2,8,121.0
8346,2009,1,26,GNB,00-0027002,Clay Matthews,False,LB,USC,23.0,2019.0,6,10,156.0
8352,2009,1,32,PIT,00-0027008,Evander Hood,False,DE,Missouri,22.0,2018.0,0,5,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12145,2023,7,249,PHI,00-0038412,Moro Ojomo,False,DE,Texas,22.0,2023.0,0,0,8.0
12146,2023,7,250,KAN,00-0038986,Nic Jones,False,CB,Ball St.,21.0,2023.0,0,0,9.0
12148,2023,7,252,BUF,00-0038646,Alex Austin,False,DB,Oregon St.,22.0,2023.0,0,0,8.0
12151,2023,7,255,SFO,00-0038648,Jalen Graham,False,S,Purdue,22.0,2023.0,0,0,4.0


In [9]:
seasonal_df = nfl.import_seasonal_data(years=[2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023], s_type='REG')
seasonal_df.info()
seasonal_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8987 entries, 0 to 8986
Data columns (total 58 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   player_id                    8987 non-null   object 
 1   season                       8987 non-null   int32  
 2   season_type                  8987 non-null   object 
 3   completions                  8987 non-null   int32  
 4   attempts                     8987 non-null   int32  
 5   passing_yards                8987 non-null   float64
 6   passing_tds                  8987 non-null   int32  
 7   interceptions                8987 non-null   float64
 8   sacks                        8987 non-null   float64
 9   sack_yards                   8987 non-null   float64
 10  sack_fumbles                 8987 non-null   int32  
 11  sack_fumbles_lost            8987 non-null   int32  
 12  passing_air_yards            8987 non-null   float64
 13  passing_yards_afte

,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr_x,special_teams_tds,fantasy_points,fantasy_points_ppr,games,tgt_sh,ay_sh,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,ppr_sh
0,00-0000108,2011,REG,1,1,14.0,1,0.0,0.0,0.0,0,0,9.0,5.0,1.0,4.223134,0,1.555556,0.000000,0,0.0,0,0.0,0.0,0.0,0.000000,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.0,4.56,4.56,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.047313
1,00-0000741,2009,REG,0,0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,0,0.0,0,0.0,0.0,0.0,0.000000,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,0.000000,0.000000,1.0,6.00,6.00,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.067568
2,00-0000865,2009,REG,1,2,17.0,0,0.0,0.0,0.0,0,0,18.0,2.0,1.0,1.449849,0,0.944444,0.000000,0,0.0,0,0.0,0.0,0.0,0.000000,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.0,0.68,0.68,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.005375
3,00-0000865,2010,REG,29,49,352.0,3,3.0,4.0,21.0,0,0,541.0,122.0,13.0,-2.543351,0,1.950153,0.369739,7,30.0,0,1.0,0.0,2.0,1.690736,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.0,23.08,23.08,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.135845
4,00-0000865,2011,REG,15,24,208.0,0,1.0,2.0,10.0,0,0,189.0,92.0,9.0,-0.096867,0,1.149171,0.154437,3,-2.0,0,0.0,0.0,0.0,0.000000,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.0,6.12,6.12,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.024270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8982,00-0039150,2023,REG,315,527,2877.0,11,10.0,62.0,477.0,9,6,4009.0,1300.0,133.0,-160.326442,1,12.786642,0.672251,39,253.0,0,2.0,0.0,18.0,21.616300,1,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.0,156.38,156.38,16,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.154486
8983,00-0039152,2023,REG,149,255,1808.0,8,4.0,28.0,185.0,5,2,2688.0,746.0,81.0,-13.886491,1,6.151852,0.462094,25,57.0,1,2.0,2.0,7.0,-9.428327,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.0,102.02,102.02,9,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.159821
8984,00-0039163,2023,REG,319,499,4108.0,23,5.0,38.0,331.0,6,3,4481.0,1762.0,188.0,64.826173,0,13.869473,1.603286,39,157.0,3,2.0,1.0,17.0,6.004496,1,1,1,0.0,0,0.0,0.0,-1.0,1.0,0.0,-0.873244,0,0.000000,0.023256,-0.003846,0.032191,0.0,274.02,275.02,15,0.001972,-0.000220,0.000562,0.002783,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.206779
8985,00-0039164,2023,REG,50,84,577.0,3,1.0,7.0,29.0,1,0,676.0,297.0,31.0,4.587791,2,5.912336,0.224194,25,136.0,4,2.0,1.0,8.0,0.430104,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.0,72.68,72.68,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.213689


In [10]:
# Keep only the desired columns in seasonal_df
columns = ['player_id', 'season', 'season_type', 'passing_epa', 'rushing_epa', 'receiving_epa', 'fantasy_points_ppr']
seasonal_df_filtered = seasonal_df[columns]

# Display the information of the filtered DataFrame
seasonal_df_filtered.info()
seasonal_df_filtered


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8987 entries, 0 to 8986
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   player_id           8987 non-null   object 
 1   season              8987 non-null   int32  
 2   season_type         8987 non-null   object 
 3   passing_epa         8987 non-null   float64
 4   rushing_epa         8987 non-null   float64
 5   receiving_epa       8987 non-null   float64
 6   fantasy_points_ppr  8987 non-null   float64
dtypes: float64(4), int32(1), object(2)
memory usage: 456.5+ KB


,player_id,season,season_type,passing_epa,rushing_epa,receiving_epa,fantasy_points_ppr
0,00-0000108,2011,REG,4.223134,0.000000,0.000000,4.56
1,00-0000741,2009,REG,0.000000,0.000000,0.000000,6.00
2,00-0000865,2009,REG,1.449849,0.000000,0.000000,0.68
3,00-0000865,2010,REG,-2.543351,1.690736,0.000000,23.08
4,00-0000865,2011,REG,-0.096867,0.000000,0.000000,6.12
...,...,...,...,...,...,...,...
8982,00-0039150,2023,REG,-160.326442,21.616300,0.000000,156.38
8983,00-0039152,2023,REG,-13.886491,-9.428327,0.000000,102.02
8984,00-0039163,2023,REG,64.826173,6.004496,-0.873244,275.02
8985,00-0039164,2023,REG,4.587791,0.430104,0.000000,72.68


In [11]:
# Merge DataFrames on gsis_id and player_id
merged_offense_df = pd.merge(filtered_offense_df, seasonal_df_filtered, left_on='gsis_id', right_on='player_id', how='inner')
merged_defense_df = pd.merge(filtered_defense_df, seasonal_df_filtered, left_on='gsis_id', right_on='player_id', how='inner')
# Display the information of the merged DataFrame
merged_offense_df.info()
merged_offense_df




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4173 entries, 0 to 4172
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   season_x            4173 non-null   int32  
 1   round               4173 non-null   int32  
 2   pick                4173 non-null   int32  
 3   team                4173 non-null   object 
 4   gsis_id             4173 non-null   object 
 5   pfr_player_name     4173 non-null   object 
 6   hof                 4173 non-null   bool   
 7   position            4173 non-null   object 
 8   college             4173 non-null   object 
 9   age                 4173 non-null   float64
 10  to                  4173 non-null   float64
 11  probowls            4173 non-null   int32  
 12  seasons_started     4173 non-null   int32  
 13  games               4173 non-null   float64
 14  player_id           4173 non-null   object 
 15  season_y            4173 non-null   int32  
 16  season

,season_x,round,pick,team,gsis_id,pfr_player_name,hof,position,college,age,to,probowls,seasons_started,games,player_id,season_y,season_type,passing_epa,rushing_epa,receiving_epa,fantasy_points_ppr
0,2009,1,1,DET,00-0026498,Matthew Stafford,False,QB,Georgia,21.0,2023.0,2,13,206.0,00-0026498,2009,REG,-73.417895,2.059116,0.000000,125.48
1,2009,1,1,DET,00-0026498,Matthew Stafford,False,QB,Georgia,21.0,2023.0,2,13,206.0,00-0026498,2010,REG,-0.056248,-0.704903,0.000000,50.50
2,2009,1,1,DET,00-0026498,Matthew Stafford,False,QB,Georgia,21.0,2023.0,2,13,206.0,00-0026498,2011,REG,100.251450,6.325917,0.000000,343.32
3,2009,1,1,DET,00-0026498,Matthew Stafford,False,QB,Georgia,21.0,2023.0,2,13,206.0,00-0026498,2012,REG,32.441723,4.277378,-0.397000,276.58
4,2009,1,1,DET,00-0026498,Matthew Stafford,False,QB,Georgia,21.0,2023.0,2,13,206.0,00-0026498,2013,REG,56.623074,-7.632060,0.000000,278.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4168,2023,6,215,LAR,00-0039136,Zach Evans,False,RB,Mississippi,22.0,2023.0,0,0,10.0,00-0039136,2023,REG,0.000000,-5.212816,0.000000,1.90
4169,2023,7,219,DET,00-0038627,Antoine Green,False,WR,North Carolina,23.0,2023.0,0,0,9.0,00-0038627,2023,REG,0.000000,0.000000,-3.538327,1.20
4170,2023,7,228,TEN,00-0038632,Colton Dowell,False,WR,UT Martin,24.0,2023.0,0,0,10.0,00-0038632,2023,REG,0.000000,0.000000,-2.652739,1.30
4171,2023,7,244,DAL,00-0038640,Jalen Brooks,False,WR,South Carolina,23.0,2023.0,0,0,7.0,00-0038640,2023,REG,0.000000,0.000000,3.854871,12.40


In [12]:
# Sort DataFrame by 'gsis_id' and 'season'
merged_offense_df_sorted = merged_offense_df.sort_values(by=['gsis_id', 'season_y'])
merged_defense_df_sorted = merged_defense_df.sort_values(by=['gsis_id', 'season_y'])

merged_offense_df_sorted.info()
merged_offense_df_sorted

merged_offense_df_sorted['position'].nunique()

<class 'pandas.core.frame.DataFrame'>
Index: 4173 entries, 0 to 4122
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   season_x            4173 non-null   int32  
 1   round               4173 non-null   int32  
 2   pick                4173 non-null   int32  
 3   team                4173 non-null   object 
 4   gsis_id             4173 non-null   object 
 5   pfr_player_name     4173 non-null   object 
 6   hof                 4173 non-null   bool   
 7   position            4173 non-null   object 
 8   college             4173 non-null   object 
 9   age                 4173 non-null   float64
 10  to                  4173 non-null   float64
 11  probowls            4173 non-null   int32  
 12  seasons_started     4173 non-null   int32  
 13  games               4173 non-null   float64
 14  player_id           4173 non-null   object 
 15  season_y            4173 non-null   int32  
 16  season_type

5

In [22]:
import numpy as np
import pandas as pd

from bokeh.io import curdoc, output_file, show
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Select, Slider, TextInput
from bokeh.plotting import figure
from bokeh.palettes import Category10_5

# Specify the filename for the HTML file
output_file("nfl_players_visualization.html")


merged_offense_df_sorted["color"] = np.where(merged_offense_df_sorted["position"] == "QB", Category10_5[0],
                                             np.where(merged_offense_df_sorted["position"] == "RB", Category10_5[1],
                                                      np.where(merged_offense_df_sorted["position"] == "WR", Category10_5[2],
                                                               np.where(merged_offense_df_sorted["position"] == "TE", Category10_5[3],
                                                                        Category10_5[4]))))
merged_offense_df_sorted["alpha"] = np.where(merged_offense_df_sorted["position"] == "QB", 0.9, 0.25)
merged_offense_df_sorted.fillna(0, inplace=True)

merged_offense_df_sorted["PPR Numbers"] = merged_offense_df_sorted['fantasy_points_ppr'].apply(lambda x: '{:,d}'.format(int(x)))

axis_map = {
    "Round": "round",
    "Passing EPA": "passing_epa",
    "Rushing EPA": "rushing_epa",
    "Receiving EPA": "receiving_epa",
    "Fantasy Points PPR": "fantasy_points_ppr",
    "Season Year": "season_y",
}

# Create Input controls
min_games = Slider(title="Minimum number of games", value=10, start=0, end=210, step=10)
season_start = Slider(title="Season Year start", start=2009, end=2023, value=2009, step=1)
season_end = Slider(title="Season Year end", start=2009, end=2023, value=2023, step=1)
min_probowls = Slider(title="Minimum number of Pro Bowls", start=0, end=9, value=0, step=1)

# Ensure that the range of Season Year End slider is dynamically set based on Season Year Start slider
season_start.js_link('value', season_end, 'start')
season_end.js_link('value', season_start, 'end')

# Drop down menu using 'position'
position_options = ["All", "QB", "RB", "WR", "TE"]
position = Select(title="Position", options=position_options, value="All")

# Create Column Data Source that will be used by the plot
source = ColumnDataSource(data=dict(x=[], y=[], color=[], title=[], year=[], alpha=[]))

TOOLTIPS = [
    ("Title", "@title"),
    ("Year", "@year"),
]

p = figure(height=600, title="", toolbar_location=None, tooltips=TOOLTIPS, sizing_mode="stretch_width")
p.circle(x="x", y="y", source=source, size=7, color="color", line_color=None, fill_alpha="alpha")

def select_players():
    selected = merged_offense_df_sorted[
        (merged_offense_df_sorted.games >= min_games.value) &
        (merged_offense_df_sorted.season_y >= season_start.value) &
        (merged_offense_df_sorted.season_y <= season_end.value) &  # Add condition for Season Year End
        (merged_offense_df_sorted.probowls >= min_probowls.value)
    ]
    if position.value != "All":
        selected = selected[selected.position == position.value]
    return selected

def update():
    df = select_players()
    x_name = axis_map[x_axis.value]
    y_name = axis_map[y_axis.value]

    p.xaxis.axis_label = x_axis.value
    p.yaxis.axis_label = y_axis.value
    p.title.text = f"{len(df)} players selected"
    source.data = dict(
        x=df[x_name],
        y=df[y_name],
        color=df["color"],
        title=df["pfr_player_name"],
        year=df["season_y"],
        alpha=df["alpha"],
    )

# Create Input controls
x_axis = Select(title="X Axis", options=sorted(axis_map.keys()), value="Round")
y_axis = Select(title="Y Axis", options=sorted(axis_map.keys()), value="Fantasy Points PPR")

controls = [min_games, season_start, season_end, min_probowls, position, x_axis, y_axis]
for control in controls:
    control.on_change('value', lambda attr, old, new: update())

inputs = column(*controls, width=320, height=800)

layout = column(row(inputs, p, sizing_mode="stretch_width"), sizing_mode="stretch_width", height=800)

update()  # initial load of the data

curdoc().add_root(layout)
curdoc().title = "NFL Players"

# Show the layout (this will also save it to the HTML file)
show(layout)


You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html

